In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T

# returns num_vars x num_samples


# starting_vals = np.array([0, 1, 2, 0, 0, 0, 1])
# ending_vals = np.array([1, 2, 3, 0, 0, 0, -1])
# out = interpMultD(starting_vals,ending_vals,10,quat_vars=[3])

# print(out)




In [11]:
########################################
# IRC BEHAVIOR 1 - with superposition ##
########################################

# Behavior three: Three passes on the IRC surface (slow with corrections)

import rospkg
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('z',ang_relative_to_surface,degrees=True)
q_ts = R_tool_surf.as_quat()


# Part 1: approach the starting point
segment1 = HybridSegment()
segment1.hybrid = False
segment1.num_samples = 600 # 2 seconds
segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.1, 0.2)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()
if q_app[3]<0:
    q_app*=-1.0

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.01, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# number of state variables by number of samples

segment1.original_vals = []
segment1.original_vals.append(orig_vals)

segments.append(segment1)


# Part 2: one pass over surface
segment2 = HybridSegment()
segment2.hybrid = True
segment2.surface = surfaceBSpline
segment2.num_samples = 2000 # 8 seconds
segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
segment2.original_vals = []

R_extra_tool = ScipyR.from_euler('y',5,degrees=True)
q_extra = R_extra_tool.as_quat()
corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))


samps = int(segment2.num_samples / 5)
app_force = -8.0
amp = 0.05
freq = 0.2
waypoint_0 = interpMultD([0.1, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
waypoint_1 = interpMultD([0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
waypoint_2 = interpMultD([0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
waypoint_3 = interpMultD([0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
waypoint_4 = interpMultD([0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])

waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

segment2.original_vals.append(waypoints)

segments.append(segment2)


# # Part 3: move away from the surface
segment3 = HybridSegment()
segment3.hybrid = False
segment3.num_samples = 200 # 2 seconds
segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment3.original_vals = []
segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.01, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
segment3.original_vals.append(orig_vals)

segments.append(segment3)


# # Part 4: move from end to start of next pass
segment4 = HybridSegment()
segment4.hybrid = False
segment4.num_samples = 400 # 4 seconds
segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment4.original_vals = []
segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()
if q_ret[3]<0:
    q_ret*=-1.0

starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
segment4.original_vals.append(orig_vals)

segments.append(segment4)

segments.extend(segments)
segments.extend(segments[:3])

QAPP:  [ 8.15450596e-04 -1.99158424e-01  9.92900872e-03  9.79916666e-01]


In [23]:
############################
# Learn IRC1 model         #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True, dt=0.01)
learnedSegments = model.learnModel(segments,config_dir+'IRC_piece1.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  18
DMP for segment  1  of  18
DMP for segment  2  of  18
DMP for segment  3  of  18
DMP for segment  4  of  18
DMP for segment  5  of  18
DMP for segment  6  of  18
DMP for segment  7  of  18
DMP for segment  8  of  18
DMP for segment  9  of  18
DMP for segment  10  of  18
DMP for segment  11  of  18
DMP for segment  12  of  18
DMP for segment  13  of  18
DMP for segment  14  of  18
DMP for segment  15  of  18
DMP for segment  16  of  18
DMP for segment  17  of  18
DMP for segment  18  of  18
Time to learn model: 151.5661838054657 seconds


[WARN] [1652223813.741185]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1652224335.731954]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1652224336.223428]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1652224336.226837]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [22]:
########################################
# The Sandening                       ##
########################################

starting_surf = [0.1, 0.2]
ending_surf = [0.1, 0.8]
num_passes = 5

import rospkg
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

from scipy.spatial.transform import Rotation as ScipyR
ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,10],degrees=True)

q_ts = R_tool_surf.as_quat()
print(q_ts)

# Part 1: approach the starting point
segment1 = HybridSegment()
segment1.hybrid = False
segment1.num_samples = 600 # 2 seconds
segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_surf[0], starting_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_app = (R_surf * R_tool_surf).as_quat()
if q_app[3]<0:
    q_app*=-1.0

print("QAPP: ",q_app)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.005, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# number of state variables by number of samples

segment1.original_vals = []
segment1.original_vals.append(orig_vals)

segments.append(segment1)


# Part 2: one pass over surface
segment2 = HybridSegment()
segment2.hybrid = True
segment2.surface = surfaceBSpline
segment2.num_samples = 3000 # 8 seconds
segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
segment2.original_vals = []

R_extra_tool = ScipyR.from_euler('y',5,degrees=True)
q_extra = R_extra_tool.as_quat()
corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))


samps = int(segment2.num_samples / 5)
app_force = -10.0
amp = 0.0
freq = 0.0

waypoint_0 = interpMultD([starting_surf[0], starting_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[ending_surf[0],ending_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],segment2.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
segment2.original_vals.append(waypoint_0)

segments.append(segment2)


# # Part 3: move away from the surface
segment3 = HybridSegment()
segment3.hybrid = False
segment3.num_samples = 200 # 2 seconds
segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment3.original_vals = []
segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()

starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
segment3.original_vals.append(orig_vals)

segments.append(segment3)


# # Part 4: move from end to start of next pass
segment4 = HybridSegment()
segment4.hybrid = False
segment4.num_samples = 400 # 4 seconds
segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
segment4.original_vals = []
segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
q_ret = (R_surf * R_tool_surf).as_quat()
if q_ret[3]<0:
    q_ret*=-1.0

starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
segment4.original_vals.append(orig_vals)

segments.append(segment4)

for ii in range(0,num_passes-2):
    segments.extend(segments[0:4])
segments.extend(segments[:3])

[-0.06162842  0.06162842  0.70441603  0.70441603]
QAPP:  [-5.30225691e-05 -1.12995201e-01  9.96229704e-03  9.93545587e-01]


In [21]:
############################
# Execute IRC1             #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([0.4091875321237706, 0.3659806127623546, 0.6255911510276543, -0.5543099010413256])
t_surf = np.array([0.34226860621143673, -0.22084696598873957, 0.281413347966159])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid
Ex Time:  0.0005350112915039062
0.0007977485656738281
Ex Time:  0.0002791881561279297
0.003930091857910156
Ex Time:  0.0002346038818359375
0.0007195472717285156
Ex Time:  0.0004839897155761719
0.0017309188842773438
Ex Time:  0.0004737377166748047
0.0007922649383544922
Ex Time:  0.0005934238433837891
0.0009381771087646484
Ex Time:  0.0008177757263183594
0.0010581016540527344
Ex Time:  0.0002722740173339844
0.0006685256958007812
Ex Time:  0.0006430149078369141
0.0008842945098876953
Ex Time:  0.0006606578826904297
0.0012600421905517578
Ex Time:  0.0008063316345214844
0.0012454986572265625
Ex Time:  0.0003972053527832031
0.0012066364288330078
Ex Time:  0.0003714561462402344
0.0008518695831298828
Ex Time:  0.0005249977111816406
0.0007724761962890625
Ex Time:  0.00045680999755859375
0.0017507076263427734
Ex Time:  0.0004405975341796875
0.0007424354553222656
Ex Time:  0.00035643577575683594
0.0008959770202636719
Ex Time:  0.000270843505859375
0.0006563663482666016
Ex Time:  0.00045

Ex Time:  0.0004405975341796875
0.0009138584136962891
Ex Time:  0.0003497600555419922
0.0021369457244873047
Ex Time:  0.0002849102020263672
0.0013782978057861328
Ex Time:  0.0002875328063964844
0.001965761184692383
Ex Time:  0.0004901885986328125
0.0009984970092773438
Ex Time:  0.0006394386291503906
0.001001596450805664
Ex Time:  0.0006814002990722656
0.0008816719055175781
Ex Time:  0.0002505779266357422
0.0008382797241210938
Ex Time:  0.0005574226379394531
0.0007207393646240234
Ex Time:  0.0002353191375732422
0.0008909702301025391
Ex Time:  0.0003809928894042969
0.0005693435668945312
Ex Time:  0.0005445480346679688
0.0014066696166992188
Ex Time:  0.00041675567626953125
0.0007517337799072266
Ex Time:  0.0014438629150390625
0.002191305160522461
Ex Time:  0.0002682209014892578
0.0009121894836425781
Ex Time:  0.0006685256958007812
0.0008666515350341797
Ex Time:  0.0004069805145263672
0.001657247543334961
Ex Time:  0.0003101825714111328
0.0007100105285644531
Ex Time:  0.0004873275756835937

Ex Time:  0.0006456375122070312
0.0017850399017333984
Ex Time:  0.0003190040588378906
0.0005724430084228516
Ex Time:  0.0002865791320800781
0.0030510425567626953
Ex Time:  0.0007476806640625
0.0009772777557373047
Ex Time:  0.00029087066650390625
0.0011065006256103516
Ex Time:  0.0006768703460693359
0.0009157657623291016
Ex Time:  0.0006284713745117188
0.0008747577667236328
Ex Time:  0.0004589557647705078
0.0007269382476806641
Ex Time:  0.00042176246643066406
0.0019609928131103516
Ex Time:  0.0004329681396484375
0.000982522964477539
Ex Time:  0.00048422813415527344
0.0011661052703857422
Ex Time:  0.0015473365783691406
0.0018057823181152344
Ex Time:  0.0003693103790283203
0.0016112327575683594
Ex Time:  0.00045180320739746094
0.0010685920715332031
Ex Time:  0.0005142688751220703
0.0011057853698730469
Ex Time:  0.0006422996520996094
0.001201629638671875
Ex Time:  0.00031113624572753906
0.0007410049438476562
Ex Time:  0.00042247772216796875
0.0007498264312744141
Ex Time:  0.000249624252319

Ex Time:  0.0005922317504882812
0.0034906864166259766
Ex Time:  0.0003809928894042969
0.0016505718231201172
Ex Time:  0.0004971027374267578
0.00077056884765625
Ex Time:  0.000598907470703125
0.0009951591491699219
Ex Time:  0.0003058910369873047
0.0010585784912109375
Ex Time:  0.0003933906555175781
0.0017888545989990234
Ex Time:  0.0006506443023681641
0.0009295940399169922
Ex Time:  0.00030612945556640625
0.0007305145263671875
Ex Time:  0.0004935264587402344
0.001050710678100586
Ex Time:  0.002117633819580078
0.0027573108673095703
Ex Time:  0.0005197525024414062
0.0010368824005126953
Ex Time:  0.00047469139099121094
0.0009548664093017578
Ex Time:  0.00048232078552246094
0.0008723735809326172
Ex Time:  0.0003230571746826172
0.0008077621459960938
Ex Time:  0.0002658367156982422
0.001363992691040039
Ex Time:  0.0003421306610107422
0.0007147789001464844
Ex Time:  0.0003039836883544922
0.0011641979217529297
Ex Time:  0.00048828125
0.0010304450988769531
Ex Time:  0.0002906322479248047
0.00116

surf calc time:  0.011571407318115234
Ex Time:  0.014539718627929688
0.014702081680297852
surf calc time:  0.009322881698608398
Ex Time:  0.010042428970336914
0.010200023651123047
surf calc time:  0.010371923446655273
Ex Time:  0.020154714584350586
0.020772218704223633
surf calc time:  0.013364553451538086
Ex Time:  0.017541170120239258
0.018546581268310547
surf calc time:  0.010149478912353516
Ex Time:  0.01755070686340332
0.018317699432373047
surf calc time:  0.014229536056518555
Ex Time:  0.0184171199798584
0.019570589065551758
surf calc time:  0.01229095458984375
Ex Time:  0.017818450927734375
0.018182039260864258
surf calc time:  0.012758970260620117
Ex Time:  0.01668572425842285
0.016897201538085938
surf calc time:  0.010910749435424805
Ex Time:  0.014152765274047852
0.014741182327270508
surf calc time:  0.009614944458007812
Ex Time:  0.016353845596313477
0.016838788986206055
surf calc time:  0.01709437370300293
Ex Time:  0.01859426498413086
0.018950700759887695
surf calc time:  

...

Ex Time:  0.0202634334564209
0.02097320556640625
surf calc time:  0.010575532913208008
Ex Time:  0.014695882797241211
0.0157620906829834
surf calc time:  0.01135706901550293
Ex Time:  0.013147115707397461
0.013395309448242188
surf calc time:  0.009992837905883789
Ex Time:  0.020905256271362305
0.0216062068939209
surf calc time:  0.011072397232055664
Ex Time:  0.013380765914916992
0.013634204864501953
surf calc time:  0.01871657371520996
Ex Time:  0.025617122650146484
0.027482271194458008
surf calc time:  0.010696887969970703
Ex Time:  0.013060808181762695
0.014303922653198242
surf calc time:  0.009735107421875
Ex Time:  0.010921716690063477
0.011134147644042969
surf calc time:  0.02748703956604004
Ex Time:  0.02928471565246582
0.03322601318359375
surf calc time:  0.009285449981689453
Ex Time:  0.010284900665283203
0.010521411895751953
surf calc time:  0.009382963180541992
Ex Time:  0.01834416389465332
0.020697593688964844
surf calc time:  0.009831905364990234
Ex Time:  0.01148867607116

surf calc time:  0.014351367950439453
Ex Time:  0.020855188369750977
0.021196842193603516
surf calc time:  0.011093616485595703
Ex Time:  0.013640642166137695
0.014441967010498047
surf calc time:  0.012371063232421875
Ex Time:  0.013722419738769531
0.013932466506958008
surf calc time:  0.009378433227539062
Ex Time:  0.010500907897949219
0.011476755142211914
surf calc time:  0.020508289337158203
Ex Time:  0.025923967361450195
0.02612137794494629
surf calc time:  0.011821985244750977
Ex Time:  0.013347625732421875
0.013712167739868164
surf calc time:  0.010131359100341797
Ex Time:  0.02079296112060547
0.02122974395751953
surf calc time:  0.009936809539794922
Ex Time:  0.012058734893798828
0.012399911880493164
surf calc time:  0.012259483337402344
Ex Time:  0.013149499893188477
0.01340627670288086
surf calc time:  0.016583919525146484
Ex Time:  0.03269052505493164
0.033309221267700195
surf calc time:  0.013955354690551758
Ex Time:  0.01878976821899414
0.019019126892089844
surf calc time: 

surf calc time:  0.009635448455810547
Ex Time:  0.012774467468261719
0.020319223403930664
surf calc time:  0.011748790740966797
Ex Time:  0.013167619705200195
0.016435861587524414
surf calc time:  0.013254642486572266
Ex Time:  0.014403581619262695
0.019126176834106445
surf calc time:  0.011259078979492188
Ex Time:  0.018126487731933594
0.019578933715820312
surf calc time:  0.013965845108032227
Ex Time:  0.01863408088684082
0.01922607421875
surf calc time:  0.010658740997314453
Ex Time:  0.011385917663574219
0.011568784713745117
surf calc time:  0.013844013214111328
Ex Time:  0.031830787658691406
0.03203868865966797
surf calc time:  0.010957002639770508
Ex Time:  0.017815113067626953
0.01918625831604004
surf calc time:  0.009081840515136719
Ex Time:  0.011321544647216797
0.01230001449584961
surf calc time:  0.013147830963134766
Ex Time:  0.016959190368652344
0.017451763153076172
surf calc time:  0.008948802947998047
Ex Time:  0.016638994216918945
0.01766204833984375
surf calc time:  0.

surf calc time:  0.018186092376708984
Ex Time:  0.02228546142578125
0.024253129959106445
surf calc time:  0.014768123626708984
Ex Time:  0.015376091003417969
0.015578031539916992
surf calc time:  0.014103412628173828
Ex Time:  0.014770746231079102
0.015688657760620117
surf calc time:  0.014633893966674805
Ex Time:  0.03057718276977539
0.030820131301879883
surf calc time:  0.011466026306152344
Ex Time:  0.013781309127807617
0.013990163803100586
surf calc time:  0.015259265899658203
Ex Time:  0.029779911041259766
0.033176422119140625
surf calc time:  0.009529829025268555
Ex Time:  0.01287221908569336
0.013080120086669922
surf calc time:  0.007569313049316406
Ex Time:  0.009990453720092773
0.010677576065063477
surf calc time:  0.008582115173339844
Ex Time:  0.011503219604492188
0.011886835098266602
surf calc time:  0.010336875915527344
Ex Time:  0.022670269012451172
0.02411818504333496
surf calc time:  0.011241912841796875
Ex Time:  0.012661933898925781
0.016048192977905273
surf calc time

KeyboardInterrupt: 